In [1]:
!pip install boto3

Defaulting to user installation because normal site-packages is not writeable


In [24]:
import boto3


In [ ]:
ec2 = boto3.client('ec2')
ec2.describe_instances()


resp = ec2.create_key_pair(KeyName="chriskeyaccess2.pem")

In [1]:
#resp

In [ ]:
file = open("creds\chriskeyaccess2.pem", "w")
file.write(resp['KeyMaterial'])
file.close

<function TextIOWrapper.close()>

CREATE A AMAZON EC2 INSTANCE THROUGH BOTO3....

In [ ]:
ec2 = boto3.client("ec2", region_name="us-east-2")
response = ec2.run_instances(
    ImageId = 'ami-060a84cbcb5c1###',
    MinCount =1,
    MaxCount = 1,
    InstanceType =  't2.micro',
    KeyName = 'chriskey#######',
    BlockDeviceMappings = [
        { 
            "DeviceName": "/dev/xvda",
            "Ebs": {
                "DeleteOnTermination": True,
                "VolumeSize": 20
            }
        }
    ]


)

In [2]:
#response

In [5]:
#ec2.describe_instances()

In [4]:
#response = ec2.describe_security_groups()
#response

In [ ]:
respnse = ec2.create_security_group(
    GroupName ='morristest',
    Description = "Security group for testing."

    
)

'respnse = ec2.create_security_group(\n    GroupName =\'morristest\',\n    Description = "Security group for testing."\n\n\n)'

In [41]:
security_group_id = respnse['GroupId'] 

security_group_id

'sg-009ff7354e20e6f05'

In [46]:
# ip. port, traffic type

response = ec2.authorize_security_group_ingress(

    GroupId = security_group_id,
    IpPermissions = [
        {
            'IpProtocol': 'tcp',
            'FromPort': 22,
            'ToPort': 22,
            'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
        }
    ]
)


In [6]:
#response

In [53]:
response = ec2.describe_instances()

In [54]:
instance_id = response['Reservations'][0]['Instances'][0]['InstanceId']

In [56]:
instance_id, security_group_id

('i-0082412cfd6ca814d', 'sg-009ff7354e20e6f05')

In [60]:
old_gid = response['Reservations'][0]['Instances'][0]['SecurityGroups'][0]['GroupId']

In [67]:
ec2.modify_instance_attribute(
    InstanceId=instance_id,         # Must be exactly "InstanceId" :contentReference[oaicite:1]{index=1}
    Groups=[security_group_id]      # Must be a list of SG IDs :contentReference[oaicite:2]{index=2}
)

{'ResponseMetadata': {'RequestId': 'a430400b-b08f-4c66-871d-5323853cf1c0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a430400b-b08f-4c66-871d-5323853cf1c0',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '235',
   'date': 'Wed, 23 Apr 2025 22:25:49 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

START AND STOP OR DELETE THE EC2 INSTANCES....



In [ ]:
import time
#response['Reservations'][0]['Instances'][0]['State']['Name']

def wait_for_status(instance_id, target_status):

    while True:
        response = ec2.describe_instances(InstanceIds=instance_id)

        status = response['Reservations'][0]['Instances'][0]['State']['Name']
        if status == target_status:
            print("Instance is in: {} state".format(target_status))
            break

        time.sleep(10)

def start_instances(instance_id):
    print("EC2 Instance start, Mac...")
    ec2.start_instances(InstanceIds= instance_id)
    

    wait_for_status(instance_id, "running....")

start_instances([instance_id])


'import time\n#response[\'Reservations\'][0][\'Instances\'][0][\'State\'][\'Name\']\n\ndef wait_for_status(instance_id, target_status):\n\n    while True:\n        response = ec2.describe_instances(InstanceIds=instance_id)\n\n        status = response[\'Reservations\'][0][\'Instances\'][0][\'State\'][\'Name\']\n        if status == target_status:\n            print("Instance is in: {} state".format(target_status))\n            break\n\n        time.sleep(10)\n\ndef start_instances(instance_id):\n    print("EC2 Instance start, Mac...")\n    ec2.start_instances(InstanceIds= instance_id)\n\n\n    wait_for_status(instance_id, "running....")\n\nstart_instances([instance_id])'

In [ ]:
def stop_instances(instance_id):
    print("EC2 Instance stopping, Mac...")
    ec2.stop_instances(InstanceIds= instance_id)

    wait_for_status(instance_id, "stopping....")
    

stop_instances([instance_id])


'def stop_instances(instance_id):\n    print("EC2 Instance stopping, Mac...")\n    ec2.stop_instances(InstanceIds= instance_id)\n\n    wait_for_status(instance_id, "stopping....")\n\n\nstop_instances([instance_id])'

In [79]:
def terminate_instances(instance_id):
    print("EC2 Instance termination...")
    ec2.terminate_instances(InstanceIds= instance_id)

    wait_for_status(instance_id, "terminated")
    

terminate_instances([instance_id])

EC2 Instance termination...
Instance is in: terminated state
